In [56]:
import math
import copy
import numpy as np
import pandas as pd
import time
from operator import itemgetter
from scipy import optimize

import matplotlib.pyplot as plt
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
import seaborn

%matplotlib inline

# Basic implementation of Nelder-Mead. 

In [61]:
# Original Paper: http://www.ime.unicamp.br/~sandra/MT853/handouts/Ref3(NelderMead1965).pdf
# Implemented from: http://www.webpages.uidaho.edu/~fuchang/res/ANMS.pdf
# and borrowed a few things from scipy
def Nelder_Mead(x0,func,
                step_size = 0.1,max_iterations=470,
                tol = 10e-6,no_improve_break=20):
    
    alpha = 1.0 # reflection
    beta  = 2.0 # contraction
    gamma = 0.5 # expansion
    
    N       = len(x0)
    x0      = np.asfarray(x0) # make sure it's float
    best    = func(x0)
    simplex = np.asfarray(np.append(x0,best))

    # for adaptive NM use
    # only use for large N
    if N > 15:
        alpha = 1.0
        beta  = 1.0 + 2.0/float(N)
        gamma = 0.75 - 1.0/(2.0*float(N))
        delta = 1.0 - 1.0/float(N)
    
    # build initial simplex
    for i in range(int(N)):
        x = np.array(x0, copy=True)
        if x[i] != 0:
            x[i] = (1 + 0.05)*x[i] # from scipy
        else:
            x[i] = 0.00025
        simplex = np.concatenate((simplex,np.append(x,func(x))))
    simplex = simplex.reshape(int(len(simplex)/(N+1)),(N+1))
    simplex = simplex[simplex[:,N].argsort()] # ascending


    no_improve = 0
    iterations = 0
    while True:
        # sort vertices by function value
        simplex = simplex[simplex[:,N].argsort()] # ascending
        min_value = simplex[0,-1]
        
        if iterations >= max_iterations and max_iterations >= max_iterations:
            print("hit max iterations")
            return simplex[0,:]
            
        iterations += 1
        
        # break after no_improv_break iterations with no improvement
        #print('...best so far:', min_value)

        if min_value < best - tol:
            no_improve = 0
            best = min_value
        else:
            no_improve += 1

        if no_improve >= no_improve_break:
            #print("Converged")
            return simplex[0,:]
        
        centroid = np.array([np.sum(simplex[:-1,i])/N for i in range(N)])
        
        # relection
        x_reflection = (1 + alpha)*centroid - alpha*simplex[-1,:-1]
        y_reflection = func(x_reflection)
        execute_shrink = 0
        
        # expansion
        if y_reflection < simplex[0,-1]:
            x_expansion = (1 + beta)*centroid - beta*simplex[-1,:-1]
            y_expansion = func(x_expansion)

            if y_expansion < y_reflection: # mistake in NL original paper at this step
                simplex[-1,:] = np.append(x_expansion,y_expansion)
                continue
            else:    
                simplex[-1,:] = np.append(x_reflection,y_reflection)
                continue
        
        else: # refelection is bigger than current min
            if  y_reflection < simplex[-2,-1]: # different than original paper
                simplex[-1,:] = np.append(x_reflection,y_reflection)
                continue
            else: # reflection is bigger than simplex[-2,-1]
                # contraction
                if y_reflection < simplex[-1,-1]:
                    x_contraction_o = -gamma*simplex[-1,:-1] + (1 + gamma)*centroid
                    y_contraction_o = func(x_contraction_o)
                    
                    if y_contraction_o <= y_reflection:
                        simplex[-1,:] = np.append(x_contraction_o,y_contraction_o)
                        continue
                    else:
                        execute_shrink = 1
                else:
                    x_contraction_i = gamma*simplex[-1,:-1] + (1 - gamma)*centroid
                    y_contraction_i = func(x_contraction_i)
                    
                    if y_contraction_i < simplex[-1,-1]:
                        simplex[-1,:] = np.append(x_contraction_i,y_contraction_i)
                        continue
                    else:
                        execute_shrink = 1
                    
                    if execute_shrink:
                        # shrink
                        low_vertex  = simplex[0,:-1]
                        new_simplex = np.zeros((N+1,N+1))
                        
                        for i,row in enumerate(simplex[:,:-1]):
                            x_new = low_vertex + (row - low_vertex)/2.0
                            y_new = func(x_new)
                            new_simplex[i,:] = np.asarray(np.append(x_new,y_new)) # maybe overwriting in loop....
                        simplex = new_simplex
                        simplex = simplex[simplex[:,N].argsort()]
                        continue


In [92]:
def func(x):
    return math.sin(x[0]) * math.cos(x[1]) * (1. / (abs(x[2]) + 1)) + 1.0

# The Rosenbrock function
def Rosen(x):
    x = np.asarray(x)
    r = np.sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0,
                  axis=0)
    return r

In [93]:
func_to_test = func

In [94]:
x0 = [-1.2,2.0,3.0]
startTime = time.time()
results_NM = Nelder_Mead(x0,func_to_test)
elapsedTime = time.time() - startTime
print("Native code: min = ",results_NM[-1],"at ",results_NM[:-1])
print("taking ",elapsedTime)

method = 'Nelder-Mead'
startTime = time.time()
result = optimize.minimize(func_to_test, x0, method=method, tol=1e-6)
elapsedTime = time.time() - startTime
print("\nScipy: min = ",func_to_test(result.x),"at ",result.x)
print("taking ",elapsedTime)

# compare results
print("\nwith a function value difference of",np.abs(func_to_test(result.x) - results_NM[-1]))
print("\nand a point differnce of",np.linalg.norm(result.x-results_NM[:-1]))

Native code: min =  1.36340079799e-05 at  [ -1.57493116e+00   3.17536909e-03   4.41492390e-08]
taking  0.0059239864349365234

Scipy: min =  7.49733608529e-13 at  [ -1.57079530e+00  -5.96649152e-07  -4.56731877e-14]
taking  0.009834051132202148

with a function value difference of 1.36340072302e-05

and a point differnce of 0.00521460549445
